In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install torch torchvision

import os
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets

In [ ]:
# Load dataset (this one for modelpth2)
path = "/content/drive/Shareddrives/AI_Stock_Predictor/data/stock_data.csv"
full_data = pd.read_csv(path)

# Display the first few rows to understand the structure
print(full_data.head())

# Parameters
sequence_length = 60
sequences = []
targets = []
scalers = {}

# Process each stock individually by its ticker
for ticker in full_data['Ticker'].unique():
    ticker_data = full_data[full_data['Ticker'] == ticker].sort_values(by='Date')

    # Initialize a MinMaxScaler for this stock's data
    scaler = MinMaxScaler(feature_range=(0, 1))
    ticker_data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(
        ticker_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    )
    scalers[ticker] = scaler  # Save the scaler for inverse transformation later

    # Create sequences and targets for each stock
    for i in range(len(ticker_data) - sequence_length):
        sequence = ticker_data.iloc[i:i + sequence_length][['Open', 'High', 'Low', 'Close', 'Volume']].values
        target = ticker_data.iloc[i + sequence_length]['Close']  # Next day's Close price
        sequences.append(sequence)
        targets.append(target)



# Convert lists to numpy arrays
sequences = np.array(sequences)
targets = np.array(targets)

         Date    Open    High      Low   Close   Volume Ticker  \
0  2024-11-12  154.53  155.04  152.370  152.64  7040557    JNJ   
1  2024-11-11  155.62  157.58  154.910  155.04  7146759    JNJ   
2  2024-11-08  157.00  157.08  155.410  155.47  8926134    JNJ   
3  2024-11-07  158.47  158.75  156.510  156.73  7522812    JNJ   
4  2024-11-06  159.90  160.85  157.355  157.88  8926601    JNJ   

        Company Name      Sector  
0  Johnson & Johnson  Healthcare  
1  Johnson & Johnson  Healthcare  
2  Johnson & Johnson  Healthcare  
3  Johnson & Johnson  Healthcare  
4  Johnson & Johnson  Healthcare  


In [ ]:
# Load dataset
path = "/content/drive/Shareddrives/AI_Stock_Predictor/data/stock_data.csv"
full_data = pd.read_csv(path)

# Display the first few rows to understand the structure
print(full_data.head())

# Parameters
sequence_length = 60
sequences = []
targets = []
scalers = {}

# Process each stock individually by its ticker
for ticker in full_data['Ticker'].unique():
    ticker_data = full_data[full_data['Ticker'] == ticker].sort_values(by='Date')

    # Drop rows with NaN values caused by moving averages
    ticker_data = ticker_data.dropna()

    # Initialize a MinMaxScaler for this stock's data
    scaler = MinMaxScaler(feature_range=(0, 1))
    ticker_data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(
        ticker_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    )
    scalers[ticker] = scaler  # Save the scaler for inverse transformation later

    # Create sequences and targets for each stock
    for i in range(len(ticker_data) - sequence_length - 5 + 1):
        sequence = ticker_data.iloc[i:i + sequence_length][['Open', 'High', 'Low', 'Close', 'Volume']].values
        print(f"sequence {i}: {sequence}")
        target = ticker_data.iloc[i + sequence_length: i + sequence_length + 5]['Close'].values  # Next day's Close price
        sequences.append(sequence)
        targets.append(target)



Streaming output truncated to the last 5000 lines.
 [0.72211508 0.71483315 0.71474984 0.720953   0.03900255]
 [0.71745419 0.74038001 0.72771288 0.75522193 0.05040438]
 [0.76840244 0.75267444 0.75898623 0.76060705 0.0510971 ]
 [0.756027   0.75650647 0.76125476 0.76631854 0.08340619]
 [0.7672774  0.75363244 0.73565275 0.7433094  0.07164283]
 [0.74108004 0.7264889  0.7375972  0.74233029 0.18438823]
 [0.739955   0.73702698 0.72317582 0.71719974 0.03855775]
 [0.71825779 0.70669008 0.70373125 0.70626632 0.07041217]
 [0.70941819 0.70285806 0.70859239 0.70251305 0.05096964]
 [0.68659595 0.67060514 0.68120796 0.67575065 0.08337138]
 [0.67518483 0.67603385 0.68234223 0.68554178 0.09127723]
 [0.68225651 0.67076481 0.67748109 0.68342037 0.08154432]
 [0.68788171 0.6832189  0.68412465 0.70267624 0.04909437]
 [0.69736419 0.68018521 0.66581435 0.65926893 0.06239987]
 [0.65348762 0.63994891 0.63089515 0.62826371 0.07482822]
 [0.61603986 0.61152802 0.62384649 0.62157311 0.06921772]
 [0.61933462 0.622385

KeyboardInterrupt: 

In [ ]:
# Convert lists to numpy arrays
sequences = np.array(sequences)
targets = np.array(targets)

# Calculate total number of sequences
num_sequences = len(sequences)
test_ratio = 0.05  # 5% test
test_size = int(num_sequences * test_ratio)

# Determine interval indices for test sets
interval_indices = [int(num_sequences * 0.2 * i) for i in range(1, 5)]

# Create training and test sets
train_sequences, train_targets = [], []
test_sequences, test_targets = [], []

for start_idx in interval_indices:
    end_idx = start_idx + test_size

    # Extract test sequences and targets for this interval
    test_sequences.append(sequences[start_idx:end_idx])
    test_targets.append(targets[start_idx:end_idx])

    # Append non-test data to training data
    train_sequences.extend(np.concatenate((sequences[:start_idx], sequences[end_idx:]), axis=0))
    train_targets.extend(np.concatenate((targets[:start_idx], targets[end_idx:]), axis=0))

In [ ]:
class StockDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.sequences[idx], dtype=torch.float32),
            torch.tensor(self.targets[idx], dtype=torch.float32)
        )

# Convert train data to numpy arrays for consistency
train_sequences = np.array(train_sequences)
train_targets = np.array(train_targets)

# Create Dataset and DataLoader for training
train_dataset = StockDataset(train_sequences, train_targets)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create DataLoaders for each test interval
test_loaders = []
for i in range(4):
    test_dataset = StockDataset(test_sequences[i], test_targets[i])
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    test_loaders.append(test_loader)

In [ ]:
sequence_len = 180
input_len = 5
hidden_size = 128
num_layers = 3
num_classes = 5
output_size = 5
learning_rate = 0.01
num_epochs = 5

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        h0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        c0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        out, _ = self.lstm(X, (h0, c0))
        out = self.fc(out[:, -1, :])  # Fully connected on the last time step
        return out

# Initialize model, loss function, and optimizer
model = LSTM(input_len, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), learning_rate)

In [ ]:
# def evaluate(model, data_loader, criterion):
#   """ Evaluates the model on a given dataset and computes the loss. """
#   model.eval() # Set model to evaluation mode
#   total_loss = 0.0
#   with torch.no_grad():
#     for sequences, targets in data_loader:
#       sequences, targets = sequences.to(torch.float32), targets.to(torch.float32) # Forward pass
#       outputs = model(sequences)
#       loss = criterion(outputs.squeeze(), targets)
#       total_loss += loss.item()
#       avg_loss = total_loss / len(data_loader)

#   return avg_loss



In [ ]:
# def train(model, criterion, optimizer, train_loader, test_loader, num_epochs):
#     model.train()
#     for epoch in range(num_epochs):
#         running_loss = 0.0
#         for sequences, targets in train_loader:
#             sequences, targets = sequences.to(torch.float32), targets.to(torch.float32)

#             # Forward pass
#             outputs = model(sequences)
#             loss = criterion(outputs.squeeze(), targets)


#             # Backward pass and optimization
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()

#             correct_predictions += (targets - 1 <= outputs <= targets + 1).sum().item()
#             total_samples += targets.size(0)
#             curr_accuracy = correct_predictions / total_samples if total_samples > 0 else 0



#         # Print loss for each epoch
#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.6f}, Accuracy: {curr_accuracy:.4f}")

# # Train the model
# train(model, criterion, optimizer, train_loader, test_loader, num_epochs)

Epoch [1/5], Loss: 0.000846
Epoch [2/5], Loss: 0.000628
Epoch [3/5], Loss: 0.000605
Epoch [4/5], Loss: 0.000596
Epoch [5/5], Loss: 0.000590


In [ ]:
# def train(model, criterion, optimizer, train_loader, test_loader, num_epochs, tolerance=1.0):
#     """
#     Trains the model and evaluates training and test accuracy after each epoch.

#     Args:
#         model (nn.Module): The LSTM model.
#         criterion: Loss function.
#         optimizer: Optimizer for backpropagation.
#         train_loader: DataLoader for training data.
#         test_loader: DataLoader for test data.
#         num_epochs (int): Number of training epochs.
#         tolerance (float): Tolerance for considering a prediction as "correct."
#     """
#     for epoch in range(num_epochs):
#         model.train()  # Set model to training mode
#         running_loss = 0.0
#         correct_predictions = 0
#         total_samples = 0

#         for sequences, targets in train_loader:
#             sequences, targets = sequences.to(torch.float32), targets.to(torch.float32)

#             # Forward pass
#             outputs = model(sequences)
#             loss = criterion(outputs.squeeze(), targets)

#             # Backward pass and optimization
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()

#             # Calculate training accuracy
#             correct_predictions += ((outputs.squeeze() >= (targets - tolerance)) &
#                                     (outputs.squeeze() <= (targets + tolerance))).sum().item()
#             total_samples += targets.size(0)

#         train_accuracy = correct_predictions / total_samples if total_samples > 0 else 0

#         # Evaluate on the test set
#         model.eval()  # Set model to evaluation mode
#         test_loss = 0.0
#         test_correct_predictions = 0
#         test_total_samples = 0
#         with torch.no_grad():
#             for sequences, targets in test_loader:
#                 sequences, targets = sequences.to(torch.float32), targets.to(torch.float32)

#                 # Forward pass
#                 outputs = model(sequences)
#                 test_loss += criterion(outputs.squeeze(), targets).item()

#                 # Calculate test accuracy
#                 test_correct_predictions += ((outputs.squeeze() >= (targets - tolerance)) &
#                                              (outputs.squeeze() <= (targets + tolerance))).sum().item()
#                 test_total_samples += targets.size(0)

#         test_accuracy = test_correct_predictions / test_total_samples if test_total_samples > 0 else 0
#         avg_train_loss = running_loss / len(train_loader)
#         avg_test_loss = test_loss / len(test_loader)

#         # Print metrics for the epoch
#         print(f"Epoch [{epoch+1}/{num_epochs}]", f"  Train Loss: {avg_train_loss:.6f}, Train Accuracy: {train_accuracy:.4f}", f"  Test Loss: {avg_test_loss:.6f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
def train(model, criterion, optimizer, train_loader, test_loader, num_epochs, tolerance=.4):
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        train_correct_predictions = 0
        train_total_samples = 0

        # Training Loop
        for sequences, targets in train_loader:
            sequences, targets = sequences.to(torch.float32), targets.to(torch.float32)

            # Forward pass
            outputs = model(sequences)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

            # Calculate training accuracy
            train_correct_predictions += ((outputs.squeeze() >= (targets - tolerance)) &
                                          (outputs.squeeze() <= (targets + tolerance))).sum().item()
            train_total_samples += targets.size(0)

        avg_train_loss = running_train_loss / len(train_loader)
        train_accuracy = train_correct_predictions / train_total_samples if train_total_samples > 0 else 0

        # Evaluate on the test set
        model.eval()
        running_test_loss = 0.0
        test_correct_predictions = 0
        test_total_samples = 0

        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences, targets = sequences.to(torch.float32), targets.to(torch.float32)

                # Forward pass
                outputs = model(sequences)
                loss = criterion(outputs.squeeze(), targets)
                running_test_loss += loss.item()

                # Calculate test accuracy
                test_correct_predictions += ((outputs.squeeze() >= (targets - tolerance)) &
                                             (outputs.squeeze() <= (targets + tolerance))).sum().item()
                test_total_samples += targets.size(0)

        avg_test_loss = running_test_loss / len(test_loader)
        test_accuracy = test_correct_predictions / test_total_samples if test_total_samples > 0 else 0

        # Print metrics for the epoch
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {avg_train_loss:.6f}, Train Accuracy: {train_accuracy/5:.4f}")
        print(f"  Test Loss: {avg_test_loss:.6f}, Test Accuracy: {test_accuracy/5:.4f}")


In [ ]:
train(model, criterion, optimizer, train_loader, test_loader, num_epochs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (5) must match the size of tensor b (32) at non-singleton dimension 1

In [ ]:
# Save the trained model weights
save_path = '/content/drive/Shareddrives/AI_Stock_Predictor/model/lstm_stock_model3.pth'
torch.save(model.state_dict(), save_path)
print("Model weights saved as Shared drive as 'lstm_stock_model3.pth'")

Model weights saved as Shared drive as 'lstm_stock_model3.pth'


In [ ]:
# Use to load weights
model = LSTM(input_len, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('/content/drive/Shareddrives/AI_Stock_Predictor/model/lstm_stock_model2.pth'))
model.eval()  # Set model to evaluation mode

<ipython-input-37-0259fd4e7335>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/Shareddrives/AI_Stock_Predictor/model/lstm_

RuntimeError: Error(s) in loading state_dict for LSTM:
	Missing key(s) in state_dict: "lstm.weight_ih_l2", "lstm.weight_hh_l2", "lstm.bias_ih_l2", "lstm.bias_hh_l2". 

In [ ]:
def predict_future_prices(model, last_sequence, days_to_predict, scaler):
    """
    Predicts future stock prices for a specified number of days.

    Args:
        model (nn.Module): Trained LSTM model.
        last_sequence (np.array): Last `sequence_length` days of data, shape (sequence_length, input_len).
        days_to_predict (int): Number of future days to predict.
        scaler (MinMaxScaler): Scaler used to normalize data for inverse transform.

    Returns:
        List[float]: Predicted stock prices for the next `days_to_predict` days.
    """
    model.eval()
    future_predictions = []
    current_sequence = last_sequence

    with torch.no_grad():
        for _ in range(days_to_predict):
            # Convert current sequence to tensor
            input_tensor = torch.tensor(current_sequence).unsqueeze(0).to(torch.float32)

            # Predict the next day
            predicted_close = model(input_tensor).item()

            # Append the prediction to the list
            future_predictions.append(predicted_close)

            # Update the sequence: shift the window and add the new prediction
            next_input = np.append(current_sequence[1:], [[0, 0, 0, predicted_close, 0]], axis=0)
            current_sequence = next_input

    # Create a dummy array to match the scaler's expected input shape
    padded_predictions = np.zeros((len(future_predictions), 5))
    padded_predictions[:, 3] = future_predictions  # Place predictions in the `Close` column

    # Inverse transform to get predictions in the original scale
    future_predictions = scaler.inverse_transform(padded_predictions)[:, 3]
    return future_predictions



In [ ]:
#trial to make farther predictions
def predict_future_prices(model, last_sequence, days_to_predict, scaler):
    """
    Predicts future stock prices for a specified number of days.

    Args:
        model (nn.Module): Trained LSTM model.
        last_sequence (np.array): Last `sequence_length` days of data, shape (sequence_length, input_len).
        days_to_predict (int): Number of future days to predict.
        scaler (MinMaxScaler): Scaler used to normalize data for inverse transform.

    Returns:
        List[float]: Predicted stock prices for the next `days_to_predict` days.
    """
    model.eval()
    future_predictions = []
    current_sequence = last_sequence

    with torch.no_grad():
        for _ in range(days_to_predict):
            # Convert current sequence to tensor
            input_tensor = torch.tensor(current_sequence).unsqueeze(0).to(torch.float32)

            # Predict the next day's Close price
            predicted_close = model(input_tensor).item()

            # Append the prediction to the list
            future_predictions.append(predicted_close)

            # Create the next input sequence
            # Use the predicted Close price and approximate other features
            next_input = current_sequence[1:]  # Shift window
            new_row = [
                predicted_close,  # Use the predicted Close price for Open
                predicted_close * 1.03,  # Approximate High (e.g., 1% above Close)
                predicted_close * 0.97,  # Approximate Low (e.g., 1% below Close)
                predicted_close,  # Use predicted Close directly
                current_sequence[-1, 4],  # Keep Volume constant (or use trend data if available)
            ]
            next_input = np.append(next_input, [new_row], axis=0)
            current_sequence = next_input

    # Create a dummy array to match the scaler's expected input shape
    padded_predictions = np.zeros((len(future_predictions), 5))
    padded_predictions[:, 3] = future_predictions  # Place predictions in the `Close` column

    # Inverse transform to get predictions in the original scale
    future_predictions = scaler.inverse_transform(padded_predictions)[:, 3]
    return future_predictions


In [ ]:
def predict_future_prices(model, last_sequence, scaler):
    """
    Predicts the stock prices for the next 5 days.

    Args:
        model (nn.Module): Trained LSTM model.
        last_sequence (np.array): Last `sequence_length` days of data.
        scaler (MinMaxScaler): Scaler used for normalization.

    Returns:
        np.array: Predicted stock prices for the next 5 days.
    """
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(last_sequence).unsqueeze(0).to(torch.float32)
        predicted_prices = model(input_tensor).squeeze().cpu().numpy()

    # Create a dummy array to match the scaler's input shape
    padded_predictions = np.zeros((5, 5))  # Assuming 5 input features
    padded_predictions[:, 3] = predicted_prices  # Place predictions in the `Close` column
    return scaler.inverse_transform(padded_predictions)[:, 3]  # Return in original scale

# Example Usage
ticker = "JNJ"
last_sequence = train_sequences[-1]  # Last training sequence for the ticker
scaler = scalers[ticker]

predicted_prices = predict_future_prices(model, last_sequence, scaler)
print("Predicted Prices for the next 5 days:", predicted_prices)

Predicted Prices for the next 5 days: [159.36647107 159.36647107 159.36647107 159.36647107 159.36647107]


In [ ]:
# Example stock: Assume we have the last 180 days of data
ticker = "AAPL"
if ticker in full_data['Ticker'].unique():
    # Extract the last 180 days for the selected ticker
    last_sequence = full_data[full_data['Ticker'] == ticker].sort_values(by='Date').iloc[-100:][['Open', 'High', 'Low', 'Close', 'Volume']].values
    last_sequence = scalers[ticker].transform(last_sequence)  # Normalize

    # Predict the next 30 days
    days_to_predict = 5
    future_prices = predict_future_prices(model, last_sequence, days_to_predict, scalers[ticker])
    print(f"Predicted prices for the next {days_to_predict} days:")
    print(future_prices)

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(future_prices) + 1), future_prices, label="Predicted Prices", color="blue")
    plt.title(f"Predicted Stock Prices for {ticker} (Next {days_to_predict} Days)")
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.legend()
    plt.show()
else:
    print(f"Ticker {ticker} not found in the dataset.")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


RuntimeError: a Tensor with 5 elements cannot be converted to Scalar